# To do

- Deal with under determined problems
- Integrate mixed-integer programing for certain foods that are discrete: eggs, bread, etc
- Convert from grams to more appropriate measure: one egg instead of 50g of eggs, etc
- Move food definition into a csv file, how would I instanceate them?
- Maybe redo constraints to avoid having to pass foods every time

In [7]:
import numpy as np
from scipy import optimize

class Food():
    def __init__(self, name, unit, serving_size, cals, carbs, proteins, fats):
        self.name = name
        self.unit = unit
        self.cals = cals/serving_size
        self.carbs = carbs/serving_size
        self.proteins = proteins/serving_size
        self.fats = fats/serving_size
    
    def __str__(self):
        return f"Name: {self.name:>20}, cals: {self.cals:.2f}/g, carbs: {self.carbs:.2f}/g, proteins: {self.proteins:.2f}/g, fats {self.fats:.2f}/g"

    def get_attr(self, attr_name):
        assert attr_name in ["cals", "carbs", "proteins", "fats", "quantity"]
        if(attr_name == "cals"):
            return self.cals
        elif(attr_name == "carbs"):
            return self.carbs
        elif(attr_name == "proteins"):
            return self.proteins
        elif(attr_name == "fats"):
            return self.fats
        else:
            return 1

class Constraint():
    def __init__(self, property, coef, b):
        assert property in ["cals", "carbs", "proteins", "fats", "quantity"]
        self.property = property
        self.coef = np.array(coef)
        self.b = b

class TotalConstraint(Constraint):
    def __init__(self, foods, property, total):
        coef = len(foods)*[1]
        super().__init__(property, coef,total)

class QuantConstraint(Constraint):
    def __init__(self, foods, food, quant):
        i = foods.index(food)
        coef = len(foods)*[0]
        coef[i]=1
        super().__init__("quantity", coef, quant)

def meal_prop(foods, constraints):
    assert type(foods) == list
    assert type(constraints) == list
    n_foods = len(foods)
    n_constraints = len(constraints)
    
    A = np.zeros((n_constraints,n_foods))
    
    b = np.zeros((n_constraints, 1))

    for i in range(n_constraints):
        constraint = constraints[i]
        A[i,:] = np.array([food.get_attr(constraint.property) for food in foods])*constraint.coef
        b[i] = constraint.b

    bounds = (0, 1000)

    x0 =np.random.uniform(low = bounds[0], high = bounds[1], size=(n_foods))

    def fun(x):
        return (A@x.reshape((-1,1)) - b).flatten()

    out = optimize.least_squares( fun = fun,x0 = x0, bounds = bounds)

    quant = out.x
    res = out.fun

    return quant, res

def get_daily_macros(bodyweight, protein_ratio, fat_ratio):
    daily_protein = bodyweight*protein_ratio
    daily_fats = fat_ratio*bodyweight

    return daily_protein, daily_fats

def get_macros_per_meal(bodyweight, protein_ratio, fat_ratio, daily_cals, n_meals):
    """Calculates the macros for each meal in a day.
    
    Recommended values are for a gaining phase.

    Args:
        Bodyweight: bodyweight in pounds.
        Protein Ratio: 1 is a standard value.
        fat_ration: values between 0.3-0.5 are recommended.
        daily_cals: Depends on experimentation and current weight gaining goal.
        n_meals: Number of meals in a day, at least 4 is recommended.

    Returns:
        cals_per_meal
        protein_per_meal
        fats_per_meal
    """

    daily_protein, daily_fats = get_daily_macros(bodyweight, protein_ratio, fat_ratio)

    cals_per_meal = daily_cals/n_meals
    protein_per_meal = daily_protein/n_meals
    fats_per_meal = daily_fats/n_meals

    return cals_per_meal, protein_per_meal, fats_per_meal

def display_goal(cals_per_meal, protein_per_meal, fats_per_meal):
    print("Goal:")
    print(f"   cals: {cals_per_meal:.0f}, proteins: {protein_per_meal:.0f}g, fats {fats_per_meal:.0f}g" )

def display_meal(foods, quantity):
    calories = 0
    carbs = 0
    proteins = 0
    fats = 0

    print("Meal:")
    for food, quant in zip(foods, quantity):
        print(f"   -{quant:.0f}g of {food.name} ")
        calories += food.cals*quant
        carbs += food.carbs*quant
        proteins += food.proteins*quant
        fats += food.fats*quant

    print("Macros:")
    print(f"   cals: {calories:.0f}, carbs: {carbs:.0f}g, proteins: {proteins:.0f}g, fats {fats:.0f}g" )


In [8]:
rice = Food("Rice", "grams", serving_size=47, cals =170, carbs=37, proteins=3, fats=0)
vegetables = Food("Vegetables", "grams", serving_size=100, cals=28, carbs=3, proteins=3, fats=0.5)
chicken = Food("Chicken breast", "grams", serving_size=112, cals=100, carbs=0, proteins=24, fats=0.5 )
oil = Food("Canola oil", "grams", serving_size=14, cals=120, carbs =0, proteins=0,fats=14)
ground_beef_85 = Food("85% ground beef", "grams", serving_size=113.4, cals = 240, carbs = 0, proteins=21, fats=17)
eggs = Food("Eggs", "grams", serving_size=50, cals =60, carbs=0, proteins=6, fats=4)
egg_whites = Food("Egg whites", "grams", serving_size=46, cals=25, carbs=0, proteins=5, fats=0)
whole_milk = Food("whole milk", "ml", serving_size=240, cals=150, carbs=12, proteins=8, fats=8 )
one_percent_millk = Food("1% milk", "ml", serving_size=240, cals=110, carbs=13, proteins=8, fats=2.5)
bread = Food( "Bread", "grams", serving_size=28, cals=70, carbs=14, proteins=3, fats=1 )

print(rice)
print(vegetables)
print(chicken)
print(oil)
print(ground_beef_85)
print(eggs)
print(egg_whites)
print(whole_milk)
print(one_percent_millk)
print(bread)

Name:                 Rice, cals: 3.62/g, carbs: 0.79/g, proteins: 0.06/g, fats 0.00/g
Name:           Vegetables, cals: 0.28/g, carbs: 0.03/g, proteins: 0.03/g, fats 0.01/g
Name:       Chicken breast, cals: 0.89/g, carbs: 0.00/g, proteins: 0.21/g, fats 0.00/g
Name:           Canola oil, cals: 8.57/g, carbs: 0.00/g, proteins: 0.00/g, fats 1.00/g
Name:      85% ground beef, cals: 2.12/g, carbs: 0.00/g, proteins: 0.19/g, fats 0.15/g
Name:                 Eggs, cals: 1.20/g, carbs: 0.00/g, proteins: 0.12/g, fats 0.08/g
Name:           Egg whites, cals: 0.54/g, carbs: 0.00/g, proteins: 0.11/g, fats 0.00/g
Name:           whole milk, cals: 0.62/g, carbs: 0.05/g, proteins: 0.03/g, fats 0.03/g
Name:              1% milk, cals: 0.46/g, carbs: 0.05/g, proteins: 0.03/g, fats 0.01/g
Name:                Bread, cals: 2.50/g, carbs: 0.50/g, proteins: 0.11/g, fats 0.04/g


In [9]:
#inputs
bodyweight = 150
n_meals = 4
protein_ratio = 1
fat_ratio = 0.4
daily_cals = 2200

cals_per_meal, protein_per_meal, fats_per_meal = get_macros_per_meal(bodyweight, protein_ratio, fat_ratio, daily_cals, n_meals)
display_goal(cals_per_meal, protein_per_meal, fats_per_meal)

Goal:
   cals: 550, proteins: 38g, fats 15g


In [10]:
foods = [rice, ground_beef_85, vegetables]

#not great having to feed foods to every constraint, maybe have a constraint glass that we feed the foods list to and then a method
#that generates the individual constraints

con_cal = TotalConstraint(foods, "cals", cals_per_meal)
con_protein = TotalConstraint(foods, "proteins", protein_per_meal)
con_fats = TotalConstraint(foods, "fats", fats_per_meal)
con_veg = QuantConstraint(foods, vegetables, 100)

constraints = [con_cal, con_protein, con_fats, con_veg]

x, res = meal_prop(foods, constraints)

cals_per_meal, protein_per_meal, fats_per_meal

display_goal(cals_per_meal, protein_per_meal, fats_per_meal)
print()
display_meal(foods, x)

Goal:
   cals: 550, proteins: 38g, fats 15g

Meal:
   -66g of Rice 
   -133g of 85% ground beef 
   -100g of Vegetables 
Macros:
   cals: 550, carbs: 55g, proteins: 32g, fats 20g


Breakfast

In [11]:
print("Current meal")
display_meal([eggs, egg_whites, whole_milk],[150, 46*2,1.5*240])

Current meal
Meal:
   -150g of Eggs 
   -92g of Egg whites 
   -360g of whole milk 
Macros:
   cals: 455, carbs: 18g, proteins: 40g, fats 24g


In [12]:
foods = [eggs, egg_whites, one_percent_millk, bread]

con_cal = TotalConstraint(foods, "cals", cals_per_meal)
con_protein = TotalConstraint(foods, "proteins", protein_per_meal)
con_fats = TotalConstraint(foods, "fats", fats_per_meal)
con_milk = QuantConstraint(foods, one_percent_millk,240)

constraints = [con_cal, con_protein, con_fats, con_milk]

x, res = meal_prop(foods, constraints)

cals_per_meal, protein_per_meal, fats_per_meal

display_goal(cals_per_meal, protein_per_meal, fats_per_meal)
print()
display_meal(foods, x)

Goal:
   cals: 550, proteins: 38g, fats 15g

Meal:
   -104g of Eggs 
   -41g of Egg whites 
   -240g of 1% milk 
   -117g of Bread 
Macros:
   cals: 550, carbs: 72g, proteins: 38g, fats 15g
